Veri Setini Yükleme

In [13]:
import pandas as pd 
import numpy as np 
import plotly.express as px

In [14]:
df = pd.read_csv('../Datasets/Manipulated Data/train.csv')
df.drop(['period'], axis=1, inplace=True)
df.head(3)

,target,gunluk_vaka,toplam_vaka,gunluk_olum,toplam_olum,gasoline_imports,gasoline_exports,crude_imports,crude_exports,gasoline_stocks,...,jet_output,propane_output,residual_fuel_output,conventional_gasoline_spot_price,crude_brent_spot_price,gasoline_future_price1,gasoline_future_price2,gasoline_future_price3,gasoline_future_price4,gasoline_demand
0,10639,0.0,0.0,0.0,0.0,779,200,8600,33,219329,...,1296,1044,414,2.202,83.11,2.139,2.119,2.128,2.146,9460
1,10036,0.0,0.0,0.0,0.0,999,200,9463,33,214942,...,1345,1021,473,2.146,81.43,2.077,2.063,2.076,2.097,10639
2,9886,0.0,0.0,0.0,0.0,871,200,8578,33,212253,...,1309,1032,401,2.122,82.25,2.098,2.072,2.085,2.106,10036


In [15]:
X = df.drop(['target'], axis=1)
y = df['target']

Splitting for Accuracy Metrics:

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, shuffle=False)

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
X_train.shape

(394, 26)

## ML Models

Bu kısımda öncelikle belirlediğim bir kaç modelin default parametrelerle birlikte nasıl bir sonuç döndürdüğünü inceleyeceğim. Devamında ise, veri setindeki `period` başlığından özellik çıkarımı ile veri setini büyütüp tekrar deneyeceğim, en yüksek performansı sergileyen modele hiper parametre ayarı yapacağım ve sonuçları raporlandıracağım. 

### Gradient Boosting
Gradient boosting, zayıf öğrenicilerin bir araya gelerek güçlü bir model oluşturduğu bir ensemble yöntemidir.

**Avantajları:**

1. Yüksek performanslı tahminler yapabilir.
1. Aşırı uyum riskini azaltabilir.
1. Çoklu öznitelikleri ve etkileşimleri yakalayabilir.

**Dezavantajları:**

1. Daha karmaşık bir model oluşturmak için daha fazla hesaplama kaynağı gerektirebilir.
1. Hiperparametrelerin doğru ayarlanması zor olabilir.
1. Büyük veri setlerinde eğitim süresi uzun olabilir.

In [19]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)

GradientBoostingRegressor()

In [20]:
gbr_preds = gbr.predict(X_test)

<hr>

### XGBoost
XGBoost, Gradient Boosting'in optimize edilmiş bir implementasyonudur.

**Avantajları:**

1. Yüksek performanslı tahminler yapabilir.
1. Aşırı uyum riskini azaltabilir ve genelleştirme yeteneğini artırabilir.
1. Öznitelik önemini değerlendirebilir ve gereksiz öznitelikleri sıfıra yakın katsayılarla düşürebilir.

**Dezavantajları:**

1. Daha fazla hesaplama kaynağı gerektirebilir ve büyük veri setlerinde eğitim süresi uzun olabilir.
1. Hiperparametrelerin doğru ayarlanması zor olabilir.
1. Veri setindeki gürültüye duyarlı olabilir.


In [21]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [22]:
xgb_preds = xgb.predict(X_test)

<hr>

### Ridge Regresyon
Ridge regresyon, L2 düzenlemesi kullanarak regresyon yapabilen bir modeldir. L2 düzeltmesi, ağırlıkların karelerinin toplamını kullanarak büyük ağırlıklara ceza verir ve modelin daha istikrarlı bir şekilde çalışmasını sağlar.

**Avantajları:**

1. Çoklu korelasyonlu öznitelikleri ele alabilir.
1. Overfitting eğilimini azaltabilir.
1. İyileştirilmiş genelleştirme performansı sunabilir.

**Dezavantajları:**
1. Öznitelik seçimi yapmak için otomatik bir mekanizma sunmaz, manuel olarak ayarlama yapılması gerekebilir.
1. Ridge regresyonunda kullanılan L2 düzenlemesi, bazı durumlarda gereğinden fazla düzleştirmeye yol açarak önemli öznitelikleri zayıflatabilir.
1. Ridge regresyonu, büyük boyutlu veri setlerinde hesaplama açısından maliyetli olabilir.

**Varsayımlar:**  
1. Lineer İlişki: Ridge regresyon, bağımlı değişken ile bağımsız değişkenler arasında bir lineer ilişki olduğunu varsayar. Bu, değişkenler arasında doğrusal bir ilişki olduğunu gösteren veriye ihtiyaç duyulduğunu gösterir.
1. Hataların Normalliği: Ridge regresyon, hataların (gözlemler ile tahminler arasındaki farklar) normal dağılım gösterdiğini varsayar. Bu, hata terimlerinin ortalama olarak sıfır olduğunu ve sabit bir varyansa sahip olduğunu gösterir.
1. Homoskedastisite: Ridge regresyon, hataların homoskedastik olduğunu varsayar, yani hataların varyansının bağımsız değişkenlerden bağımsız olduğunu gösterir. Bu, hataların gözlemler arasında eşit bir şekilde dağıldığı anlamına gelir.
1. Bağımsızlık: Ridge regresyon, gözlemlerin birbirinden bağımsız olduğunu varsayar. Yani, gözlemlerin birbirini etkilemediği ve aralarında bağımsız olduğu kabul edilir.
1. Aykırı Değerlerin Etkisi: Ridge regresyon, aykırı değerlerin modele büyük bir etkisi olduğunu varsaymaz. Aykırı değerlerin modele olan etkisini azaltmada L2 düzeltmesi (Ridge düzeltmesi) yardımcı olabilir.

**Not:**  
Yukarıdaki varsayımları `../Notebooks/3-Regression-Study.ipynb` kısmında çalıştım. Ancak ilgili notebook'a baktığımızda aykırı değer haricinde genel anlamda varsayımların sağlandığını görebiliriz. Bu nedenle Ridge Regresyon Modelini de deneyeceğim.

In [23]:
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(X_train, y_train)

Ridge()

In [24]:
ridge_preds = ridge.predict(X_test)

<hr>

### Rassal Ormanlar
Rassal ormanlar, karar ağaçlarının bir araya getirilerek oluşturulan bir ensemble yöntemidir.

**Avantajları:**

1. Daha güçlü ve karmaşık modeller oluşturabilir.
1. Overfitting riskini azaltabilir.
1. Veri setindeki özniteliklerin önemini değerlendirebilir.

**Dezavantajları:**

1. Eğitim süreci daha yavaş olabilir, çünkü birden fazla ağaç eğitilmelidir.
1. Veri setindeki gürültüye duyarlı olabilir.
1. Ağaçların birleştirilmesi ve sonuçların yorumlanması karmaşık olabilir.

In [36]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [37]:
rf_preds = rf.predict(X_test)

<hr>

## Modellerin Değerlendirilmesi

In [25]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

In [33]:
rmse = np.sqrt(mean_squared_error(y_test, gbr_preds))
mape = mean_absolute_percentage_error(y_test, gbr_preds)
mae = mean_absolute_error(y_test, gbr_preds)

print(f"""Gradient Boosting Regressor
---------------------------
• RMSE: {rmse}
• MAE: {mae}
• MAPE: {mape}
      """)

Gradient Boosting Regressor
---------------------------
• RMSE: 733.0976242349232
• MAE: 561.0100104936284
• MAPE: 0.059402291531281355
      


In [34]:
rmse = np.sqrt(mean_squared_error(y_test, xgb_preds))
mape = mean_absolute_percentage_error(y_test, xgb_preds)
mae = mean_absolute_error(y_test, xgb_preds)

print(f"""XGBoost Regressor
---------------------------
• RMSE: {rmse}
• MAE: {mae}
• MAPE: {mape}
      """)

XGBoost Regressor
---------------------------
• RMSE: 809.3880235587814
• MAE: 626.9899384469697
• MAPE: 0.06460242357571126
      


In [35]:
rmse = np.sqrt(mean_squared_error(y_test, ridge_preds))
mape = mean_absolute_percentage_error(y_test, ridge_preds)
mae = mean_absolute_error(y_test, ridge_preds)

print(f"""Ridge Regressor
---------------------------
• RMSE: {rmse}
• MAE: {mae}
• MAPE: {mape}
      """)

Ridge Regressor
---------------------------
• RMSE: 838.6468739377128
• MAE: 600.173803877967
• MAPE: 0.06599317260858023
      


In [39]:
rmse = np.sqrt(mean_squared_error(y_test, rf_preds))
mape = mean_absolute_percentage_error(y_test, rf_preds)
mae = mean_absolute_error(y_test, rf_preds)

print(f"""Random Forest Regressor
---------------------------
• RMSE: {rmse}
• MAE: {mae}
• MAPE: {mape}
      """)

Random Forest Regressor
---------------------------
• RMSE: 669.3270362261129
• MAE: 485.97505050505055
• MAPE: 0.051335886703692137
      


<h4 align="center">Temel Veri Seti Sonuçları</h4>


<div align="center">

|Model                      |RMSE  |MAE   |MAPE |
|:--------------------------|:----:|:----:|:---:|
|Random Forest Regressor|669.32|485.97|0.051|
|Gradient Boosting Regressor|733.09|561.01|0.059|
|XGBoost Regressor|809.38|626.98|0.064|
|Ridge Regressor|838.64|600.17|0.065|

</div>

**Not:**  
Tahmin edilmesi istenilen hedef değişkenin ortalaması: 9635.77

<hr>

## Zaman Özellik Çıkarımı ile Veri Setinin Büyütülmesi

In [78]:
df = pd.read_csv('../Datasets/Manipulated Data/train.csv')
df.head(3)

,period,target,gunluk_vaka,toplam_vaka,gunluk_olum,toplam_olum,gasoline_imports,gasoline_exports,crude_imports,crude_exports,...,jet_output,propane_output,residual_fuel_output,conventional_gasoline_spot_price,crude_brent_spot_price,gasoline_future_price1,gasoline_future_price2,gasoline_future_price3,gasoline_future_price4,gasoline_demand
0,2010-10-15,10639,0.0,0.0,0.0,0.0,779,200,8600,33,...,1296,1044,414,2.202,83.11,2.139,2.119,2.128,2.146,9460
1,2010-10-22,10036,0.0,0.0,0.0,0.0,999,200,9463,33,...,1345,1021,473,2.146,81.43,2.077,2.063,2.076,2.097,10639
2,2010-10-29,9886,0.0,0.0,0.0,0.0,871,200,8578,33,...,1309,1032,401,2.122,82.25,2.098,2.072,2.085,2.106,10036


In [79]:
df['period'] = pd.to_datetime(df['period'])

df['year'] = df['period'].dt.year
df['month'] = df['period'].dt.month
df['week'] = df['period'].dt.isocalendar().week
df['day_of_year'] = df['period'].dt.dayofyear
df['day_of_month'] = df['period'].dt.day
df['quarter'] = df['period'].dt.quarter

df[['period','year', 'month', 'week',
    'day_of_year','day_of_month', 'quarter']].head(3)

,period,year,month,week,day_of_year,day_of_month,quarter
0,2010-10-15,2010,10,41,288,15,4
1,2010-10-22,2010,10,42,295,22,4
2,2010-10-29,2010,10,43,302,29,4


In [80]:
import holidays
from datetime import datetime, timedelta

def haftada_tatil_gunu_sayisi(tarih):

    tatil_takvimi = holidays.Turkey()

    hafta_numarasi = tarih.isocalendar()[1]
    
    baslangic_tarihi = tarih - timedelta(days=tarih.weekday())
    bitis_tarihi = baslangic_tarihi + timedelta(days=6)
    
    tatil_sayisi = 0
    for gun in range((bitis_tarihi - baslangic_tarihi).days + 1):
        kontrol_tarihi = baslangic_tarihi + timedelta(days=gun)
        if kontrol_tarihi in tatil_takvimi:
            tatil_sayisi += 1
    
    return tatil_sayisi

In [81]:
df['Haftada_Tatil_Sayisi'] = df['period'].apply(haftada_tatil_gunu_sayisi)

df[['period','year', 'month', 'week', 'day_of_year','day_of_month', 'quarter', 'Haftada_Tatil_Sayisi']].head(3)

,period,year,month,week,day_of_year,day_of_month,quarter,Haftada_Tatil_Sayisi
0,2010-10-15,2010,10,41,288,15,4,0
1,2010-10-22,2010,10,42,295,22,4,0
2,2010-10-29,2010,10,43,302,29,4,1


In [82]:
X = df.drop(['target', 'period'], axis=1)
y = df['target']

Veri Setini Ayırma

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, shuffle=False)

Scale Etme

In [84]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [87]:
X_train.shape, X_test.shape, type(X_train), type(X_test)

((394, 33), (99, 33), numpy.ndarray, numpy.ndarray)

Modeller:  
* XGBoost Regressor

In [92]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_test)

* Gradient Boosting Regressor

In [93]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)
gbr_preds = gbr.predict(X_test)

* Random Forest Regressor

In [94]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)

* Ridge Regressor

In [95]:
rr = Ridge()
rr.fit(X_train, y_train)
rr_preds = rr.predict(X_test)

Model Accuracy

In [96]:
rmse = np.sqrt(mean_squared_error(y_test, xgb_preds))
mape = mean_absolute_percentage_error(y_test, xgb_preds)
mae = mean_absolute_error(y_test, xgb_preds)

print(f"""XGBoost Regressor
---------------------------
• RMSE: {rmse}
• MAE: {mae}
• MAPE: {mape}
        """)

XGBoost Regressor
---------------------------
• RMSE: 674.7007223911343
• MAE: 472.4308120265151
• MAPE: 0.05025267224251711
        


In [97]:
rmse = np.sqrt(mean_squared_error(y_test, gbr_preds))
mape = mean_absolute_percentage_error(y_test, gbr_preds)
mae = mean_absolute_error(y_test, gbr_preds)

print(f"""Gradient Boosting Regressor
---------------------------
• RMSE: {rmse}
• MAE: {mae}
• MAPE: {mape}
        """)

Gradient Boosting Regressor
---------------------------
• RMSE: 649.9799958838482
• MAE: 492.16764691244924
• MAPE: 0.05208734718301816
        


In [98]:
rmse = np.sqrt(mean_squared_error(y_test, rf_preds))
mape = mean_absolute_percentage_error(y_test, rf_preds)
mae = mean_absolute_error(y_test, rf_preds)

print(f"""Random Forest Regressor
---------------------------
• RMSE: {rmse}
• MAE: {mae}
• MAPE: {mape}
        """)

Random Forest Regressor
---------------------------
• RMSE: 621.0930718499442
• MAE: 448.7794949494949
• MAPE: 0.047459161299232284
        


In [99]:
rmse = np.sqrt(mean_squared_error(y_test, rr_preds))
mape = mean_absolute_percentage_error(y_test, rr_preds)
mae = mean_absolute_error(y_test, rr_preds)

print(f"""Ridge Regressor
---------------------------
• RMSE: {rmse}
• MAE: {mae}
• MAPE: {mape}
        """)

Ridge Regressor
---------------------------
• RMSE: 887.1535576581365
• MAE: 643.0609728055224
• MAPE: 0.07071931171979946
        


<div align="center" class="columns" markdown="1">

Tablo: Temel Veri Seti Sonuçları

|Model                      |RMSE  |MAE   |MAPE |
|:--------------------------|:----:|:----:|:---:|
|Random Forest Regressor|669.32|485.97|0.051|
|Gradient Boosting Regressor|733.09|561.01|0.059|
|XGBoost Regressor|809.38|626.98|0.064|
|Ridge Regressor|838.64|600.17|0.065|

<br>
<hr width=450>

Tablo: Özellik Çıkarımı ile Genişletilmiş Veri Seti Sonuçları

|Model                      |RMSE  |MAE   |MAPE |
|:--------------------------|:----:|:----:|:---:|
|Random Forest Regressor|621.09| 448.77|0.047|
|Gradient Boosting Regressor|649.97|492.16|0.052|
|XGBoost Regressor|674.70|472.43|0.050|
|Ridge Regressor|887.15|643.06|0.070|

<br>
<hr width=450>

**Not:**  
Tahmin edilmesi istenilen hedef değişkenin ortalaması: 9635.77

</div>


<hr>

### Hiper Parametre Optimizasyonu

In [106]:
model = RandomForestRegressor(criterion='friedman_mse')

In [107]:
param_grid = {'n_estimators': [100, 200, 500],
        'max_depth': [3, 5, 7],
        'max_features': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
        }

In [109]:
from sklearn.model_selection import GridSearchCV
import time 
import warnings
warnings.filterwarnings("ignore")

start = time.time()

grid = GridSearchCV(model, param_grid, cv=5, verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)

best_params = grid.best_params_

print("Best Parameters:", best_params)
print("-"*50)

end = time.time()
total_time = end - start
minutes = int(total_time // 60)
seconds = int(total_time % 60)
print(f"Toplam Optimizasyon Süresi: {minutes} dakika {seconds} saniye")

Fitting 5 folds for each of 486 candidates, totalling 2430 fits
Best Parameters: {'bootstrap': False, 'max_depth': 7, 'max_features': 7, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
------------------------------
Best RMSE: nan
------------------------------
Toplam Optimizasyon Süresi: 6 dakika 56 saniye


In [216]:
model = RandomForestRegressor(n_estimators=100, max_depth=20,
                            max_features=7, min_samples_leaf=1,
                            min_samples_split=2, bootstrap=0)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=0, max_depth=20, max_features=7)

In [217]:
y_pred = model.predict(X_test)

In [218]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"""Tuned Random Forest Regressor
---------------------------
• RMSE: {rmse}
• MAE: {mae}
• MAPE: {mape}
        """)

Tuned Random Forest Regressor
---------------------------
• RMSE: 618.9687814395393
• MAE: 441.88061808561804
• MAPE: 0.04638358550485433
        


In [219]:
model = RandomForestRegressor(n_estimators=100, max_depth=20,
                            max_features=7, min_samples_leaf=1,
                            min_samples_split=2, bootstrap=0)
model.fit(X, y)

RandomForestRegressor(bootstrap=0, max_depth=20, max_features=7)

In [221]:
import pickle

# Eğitilmiş modeli kaydetmek
with open('../Notebooks/Models/rf_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Kaydedilen modeli yükleme
# with open('../Notebooks/Models/rf_model.pkl', 'rb') as f:
#     model = pickle.load(f)

In [222]:
from joblib import load, dump

# Eğitilmiş modeli kaydetmek
dump(model, '../Notebooks/Models/rf_model.joblib')


# Kaydedilen modeli yükleme
# model = load('../Notebooks/Models/rf_model.joblib')

['../Notebooks/Models/rf_model.joblib']

In [224]:
def dataset_preparer(df):
    import pandas as pd 
    import numpy as np
    
    df['period'] = pd.to_datetime(df['period'])

    df['year'] = df['period'].dt.year
    df['month'] = df['period'].dt.month
    df['week'] = df['period'].dt.isocalendar().week
    df['day_of_year'] = df['period'].dt.dayofyear
    df['day_of_month'] = df['period'].dt.day
    df['quarter'] = df['period'].dt.quarter

    import holidays
    from datetime import datetime, timedelta

    def haftada_tatil_gunu_sayisi(tarih):

        tatil_takvimi = holidays.Turkey()

        hafta_numarasi = tarih.isocalendar()[1]
        
        baslangic_tarihi = tarih - timedelta(days=tarih.weekday())
        bitis_tarihi = baslangic_tarihi + timedelta(days=6)
        
        tatil_sayisi = 0
        for gun in range((bitis_tarihi - baslangic_tarihi).days + 1):
            kontrol_tarihi = baslangic_tarihi + timedelta(days=gun)
            if kontrol_tarihi in tatil_takvimi:
                tatil_sayisi += 1
        
        return tatil_sayisi
    
    df['Haftada_Tatil_Sayisi'] = df['period'].apply(haftada_tatil_gunu_sayisi)
    
    print("Veri Setini Standartlaştırarak Modeli .fit() Etmeyi Unutmayınız...")
    
    return df